In [1]:
import subprocess as sp
import numpy as np
import k3d

### Test

In [ ]:
plot = k3d.plot()
plot += k3d.line([[0, 0, 0],
                 [1, 1, 1]])
plot.display()

### Trial 1

In [2]:
def getFacets3D(filename):
    exe = ["./getFacets3D", filename]
    p = sp.Popen(exe, stdout=sp.PIPE, stderr=sp.PIPE)
    stdout, stderr = p.communicate()
    temp1 = stderr.decode("utf-8")

    lines = temp1.strip().split('\n')

    all_points = []
    all_indices = []
    current_index = 0

    for line in lines:
        if line.strip() == '':
            # Move to next plane
            if len(all_points) > current_index:
                # Calculate indices for the previous plane
                n = len(all_points) - current_index
                indices = np.vstack((np.zeros(n-2, dtype=np.uint32) + current_index,
                                     np.arange(1, n-1, dtype=np.uint32) + current_index,
                                     np.arange(2, n, dtype=np.uint32) + current_index)).T.flatten()
                all_indices.append(indices)
                current_index = len(all_points)
        else:
            # Convert each line into a tuple of floats (x, y, z) and add it to all_points
            point = tuple(map(float, line.split()))
            all_points.append(point)

    # Adding indices for the last plane if it exists
    if len(all_points) > current_index:
        n = len(all_points) - current_index
        indices = np.vstack((np.zeros(n-2, dtype=np.uint32) + current_index,
                             np.arange(1, n-1, dtype=np.uint32) + current_index,
                             np.arange(2, n, dtype=np.uint32) + current_index)).T.flatten()
        all_indices.append(indices)

    # Convert lists to numpy arrays
    all_points = np.array(all_points, dtype=np.float32)
    all_indices = np.concatenate(all_indices).astype(np.uint32)

    return all_points, all_indices

In [4]:
# Example usage
points, indices = getFacets3D("dump")
print(len(points))
print(len(indices))

152748
229650


In [11]:
# Plotting with k3d
plot = k3d.plot()
mesh = k3d.mesh(points, indices, wireframe=True)  # Set opacity to 1 for full solidity
plot += mesh
plot.display()


Output()